In [1]:
import ipywidgets as w

from ldap3 import Server, Connection, MODIFY_REPLACE
from passlib.hash import ldap_salted_sha1

In [4]:
def change_password(user_name, old_pass, new_pass, new_pass2):
    dn = 'cn={},ou=users,dc=pl,dc=eso,dc=org'.format(user_name)
    server = Server('ldap://ldap-server')
    connection = Connection(server, 'cn=admin,dc=pl,dc=eso,dc=org', 'admin', auto_bind=True)
    connection.search('ou=users,dc=pl,dc=eso,dc=org', '(cn={})'.format(user_name), attributes=['userPassword'])

    if len(connection.entries) == 0:
        return 'User does not exist.'

    current_pass = connection.entries[0].entry_attributes_as_dict['userPassword'][0]
    if not ldap_salted_sha1.verify(old_pass, current_pass):
        return 'Old password incorrect.'
    
    if new_pass != new_pass2:
        return 'Passwords do not match.'

    if connection.modify(dn, {
        'userPassword': [(MODIFY_REPLACE, [ldap_salted_sha1.hash(new_pass)])]
    }):
        return 'Success.'
    return 'LDAP error.'

## Change your Datalab password
---

In [3]:
w_username = w.Text(placeholder='User name')
w_old_password = w.Password(placeholder='Current password')
w_new_password = w.Password(placeholder='New password')
w_new_password2 = w.Password(placeholder='Repeat new password')
w_mod_pass_button = w.Button(description='Change Password', icon='key', button_style='success')
w_mod_pass_result = w.Label()

def button_mod_pass_action(*_):
    w_mod_pass_result.value = change_password(
        w_username.value, w_old_password.value, w_new_password.value, w_new_password2.value
    )

w_mod_pass_button.on_click(button_mod_pass_action)
w.VBox([w_username, w_old_password, w_new_password, w_new_password2, w_mod_pass_button, w_mod_pass_result])